In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
%matplotlib inline


from sklearn.model_selection import train_test_split, ShuffleSplit
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer 
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectPercentile
from sklearn import set_config
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.feature_selection import SequentialFeatureSelector

import optuna
import statsmodels.api as sm
from optuna.distributions import CategoricalDistribution, IntDistribution, FloatDistribution
from optuna.integration import OptunaSearchCV, ShapleyImportanceEvaluator

from sklearn.experimental import enable_iterative_imputer  # still experimental 
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer 
import category_encoders as ce

plt.rc('font', family='Malgun Gothic') # 폰트 지정
plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정
%config InlineBackend.figure_format='retina' # 그래프 글씨 뚜렷

In [2]:
X_train = pd.read_csv('X_train.csv', encoding='cp949').drop(columns='ID')
y_train = pd.read_csv('y_train.csv', encoding='cp949').Salary

X_test = pd.read_csv('X_test.csv', encoding='cp949')
test_id = X_test.ID
X_test = X_test.drop(columns='ID')

In [3]:
# 전처리 과정까지 처리한 데이터 불러오기
X = pd.read_csv('pre_processing.csv', encoding='cp949')

# Featre Generation

- 숙련도 >> v1 사용

In [4]:
#근무경력에 따른 숙련도 칼럼 생성

X['숙련도'] = X['근무경력_y']

# v1
X['숙련도'] = X['숙련도'].apply(lambda x:'경력없음' if x==0 else '하' if x<= 4 else '중' if 4<x<=10 else '상')

# v2
# X['숙련도'] = X['숙련도'].apply(lambda x: '하' if 0<=x<=2.5 else '중' if 2.5<x<= 4 else '상' if 4<x<=10 else '최상')

# v3
# X['숙련도'] = X['숙련도'].apply(lambda x:'경력없음' if x==0 else '하' if x<= 4 else '중' if 4<x<=10 else '상' if 10<x<=14 else '극상')

X['숙련도'].value_counts()

경력없음    15690
하        5247
중        4867
상        1814
Name: 숙련도, dtype: int64

- 해외근무경력 여부(유, 무) feature 생성

In [5]:
region_lst=[]
for val in X['근무지역']:
    region_lst.append(val.split(','))

foreign = ['홍콩','미국','해외','중국','캐나다','인도네시아','일본','싱가포르','필리핀','대만','말레이시아','인도네시아','러시아','인도'
          ,'프랑스','필리핀','말레이시아','방글라데시']

X['해외근무경력'] = np.zeros(shape=(X.shape[0],), dtype=np.object)
for k in range(0, len(region_lst)):
    for val in region_lst[k]:
        if val in foreign:
            X['해외근무경력'][k] = '1'
            
X['해외근무경력'] = X['해외근무경력'].map(lambda x: '유' if x == '1' else '무')

- 직무태그_pr

In [6]:
# 값들 사이, 앞, 뒤 공백 제거
X['직무태그_pr'] = X['직무태그'].map(lambda x: x.replace(' ', ''))

# 양쪽 콤마 제거
X['직무태그_pr'] = X['직무태그'].map(lambda x: x.strip(','))

# 양쪽 온점 제거
X['직무태그_pr'] = X['직무태그'].map(lambda x: x.strip('.'))

# 콤마를 기준으로 split 후 첫 번째 단어만 뽑아서 '직무태그_pr' feature 생성
X['직무태그_pr'] = X['직무태그'].map(lambda x: x.split(',')[0])

In [7]:
X['직무태그_pr'].head(5)

0     취재기자
1       없음
2    하드웨어 
3      반도체
4      기술직
Name: 직무태그_pr, dtype: object

In [8]:
# '직무태그_pr' feature value의 첫번째, 두번째 글자만 인덱싱해서 그룹화 
X['직무태그_pr'] = X['직무태그_pr'].str[:2]

In [9]:
X['직무태그_pr'].head(5)

0    취재
1    없음
2    하드
3    반도
4    기술
Name: 직무태그_pr, dtype: object

- 대학별그룹

In [10]:
X['대학별그룹'] = X['출신대학'] 

top_univ = ['서울과학기술대학교', '성균관대학교','연세대학교','중앙대학교','이화여자대학교', '동아대학교']
high_univ = ['경기대학교', '수원대학교','세종대학교','인천대학교']
middle_univ = ['한림대학교', '상지대학교', '한성대학교', '동의대학교','제주대학교','한국산업기술대학교']
low_univ = ['신라대학교','한밭대학교','전주대학교','군산대학교','성결대학교','호남대학교','세명대학교', '용인대학교',
           '광주대학교','서원대학교','동서대학교','한일장신대학교','한세대학교','협성대학교','서울신학대학교','송원대학교','부산디지털대학교',
           '선문대학교','성공회대학교','호원대학교', '남부대학교', '서울여자대학교', '성신여자대학교', '동덕여자대학교', '경주대학교','목포대학교']

a = X[X['대학별그룹'].isin(top_univ)]['대학별그룹']
X['대학별그룹'] = X['대학별그룹'].replace(list(a),'top_univ')

a = X[X['대학별그룹'].isin(high_univ)]['대학별그룹']
X['대학별그룹'] = X['대학별그룹'].replace(list(a),'high_univ')

a = X[X['대학별그룹'].isin(middle_univ)]['대학별그룹']
X['대학별그룹'] = X['대학별그룹'].replace(list(a),'middle_univ')

a = X[X['대학별그룹'].isin(low_univ)]['대학별그룹']
X['대학별그룹'] = X['대학별그룹'].replace(list(a),'low_univ')

In [11]:
X['대학별그룹'].head(5)

0     top_univ
1     low_univ
2    high_univ
3    high_univ
4     low_univ
Name: 대학별그룹, dtype: object

- 출신대학별 근무경력 mean & std & var

In [12]:
X['대학_평균근무경력'] = X.groupby(['출신대학'])['근무경력_d'].transform('mean')
X['대학_근무경력std'] = X.groupby(['출신대학'])['근무경력_d'].transform('std')
X['대학_근무경력var'] = X.groupby(['출신대학'])['근무경력_d'].transform('var')

- 세부직종별 근무경력 mean & std & var

In [13]:
X['세부직종_평균근무경력'] = X.groupby(['세부직종'])['근무경력_d'].transform('mean')
X['세부직종_근무경력std'] = X.groupby(['세부직종'])['근무경력_d'].transform('std')
X['세부직종_근무경력var'] = X.groupby(['세부직종'])['근무경력_d'].transform('var')

- 직종별 근무경력 mean & std & var

In [14]:
X['직종_평균근무경력'] = X.groupby(['직종'])['근무경력_d'].transform('mean')
X['직종_근무경력std'] = X.groupby(['직종'])['근무경력_d'].transform('std')
X['직종_근무경력var'] = X.groupby(['직종'])['근무경력_d'].transform('var')

- 근무형태별 근무경력 mean & std & var

In [15]:
X['근무형태_평균근무경력'] = X.groupby(['근무형태'])['근무경력_d'].transform('mean')
X['근무형태_근무경력std'] = X.groupby(['근무형태'])['근무경력_d'].transform('std')
X['근무형태_근무경력var'] = X.groupby(['근무형태'])['근무경력_d'].transform('var')

- 대학전공별 근무경력 mean & std & var

In [16]:
X['대학전공_평균근무경력'] = X.groupby(['대학전공'])['근무경력_d'].transform('mean')
X['대학전공_근무경력std'] = X.groupby(['대학전공'])['근무경력_d'].transform('std')
X['대학전공_근무경력var'] = X.groupby(['대학전공'])['근무경력_d'].transform('var')

- 직무태그_pr별 근무경력 mean & std & var

In [17]:
X['직무태그_pr_평균근무경력'] = X.groupby(['직무태그_pr'])['근무경력_d'].transform('mean')
X['직무태그_pr_근무경력std'] = X.groupby(['직무태그_pr'])['근무경력_d'].transform('std')
X['직무태그_pr_근무경력var'] = X.groupby(['직무태그_pr'])['근무경력_d'].transform('var')

- 세부직종 연봉기준 정렬하여 순위(order) 부여

In [18]:
X_train_c = X_train.copy()
X_train_c['Salary'] = y_train

job_lst = X['세부직종'].unique().tolist()
job_salary = []

for job in job_lst:
    mean_salary = X_train_c['Salary'][X['세부직종'] == job].mean()
    job_salary.append(mean_salary)
    
job_mean_salary = pd.DataFrame({'세부직종' : job_lst,
                                 '평균연봉' : job_salary})
job_mean_salary = job_mean_salary.fillna(job_mean_salary.평균연봉.mean()) ; job_mean_salary.sort_values('평균연봉', ascending=True)

,세부직종,평균연봉
87,정밀공학·광학,1700.000000
60,정보검색·정보제공,1750.000000
43,판매·매장관리·캐셔·프런트,1810.344828
63,캐릭터·애니메이션,1857.142857
75,사서·문서관리,1860.000000
...,...,...
85,기타 건설·기계·전기·전자,4628.571429
78,재무·IR·자금·감사,4713.888889
77,세무사·회계사·관세사,4864.285714
104,연예·공연기획·매니저,5250.000000


In [19]:
# 세부직종 정렬 리스트 목록 저장
tra_1 = job_mean_salary.sort_values('평균연봉', ascending=True)['세부직종'].tolist()

# 정렬 리스트에 순차적으로 번호(order) 부여
job_order_dict = {i : j for j, i in enumerate(tra_1)}

# '세부직종_order' feature 생성
X['세부직종_order'] = X['세부직종'].map(job_order_dict)

- 출신대학 그룹화하여 수치 부여한 '출신대학수치' & '출신대학수치 * 대학성적'

In [20]:
top_rank = ['성균관대학교', '연세대학교', '중앙대학교', '이화여자대학교']
mid_rank = ['세종대학교', '성신여자대학교']
low_rank = ['한성대학교', '동덕여자대학교', '서울여자대학교', '서울과학기술대학교']
province = ['제주대학교', '인천대학교', '군산대학교', '한국산업기술대학교']
etc= ['수원대학교', '전주대학교', '세명대학교', '신라대학교', '상지대학교', '한밭대학교', '경기대학교', '선문대학교', '성공회대학교', '호원대학교', '한림대학교', '목포대학교', '동아대학교', '동의대학교', '성결대학교', '호남대학교', '광주대학교', '서원대학교', '동서대학교', '한일장신대학교', '한세대학교', '용인대학교', '경주대학교', '협성대학교', '서울신학대학교', '송원대학교', '부산디지털대학교','남부대학교']


university = []
for i in X['출신대학']:
    if i in top_rank:
        university.append(5)
    elif i in mid_rank:
        university.append(4)
    elif i in low_rank:
        university.append(3)
    elif i in province:
        university.append(2)
    elif i in etc:
        university.append(1)
        
# '출신대학수치' feature
X['출신대학수치'] = university

# 출신대학수치 * 대학성적 feature
X['출신대학수치*대학성적'] = X['출신대학수치'] * X['대학성적']

- '출신대학수치'와 유사한 방법으로 '근무경력수치'  feature 생성

In [21]:
career = []
# v1.0
for i in X['근무경력_y']:
    if i == 0:
        career.append(0)
    elif 0 < i < 3:
        career.append(1)
    elif 3 <= i < 6:
        career.append(2)
    elif 6 <= i < 10:
        career.append(3)
    elif 10 <= i < 15:
        career.append(4)
    elif 15 <= i < 20:
        career.append(5)
    elif 20 <= i < 25:
        career.append(6)
    elif 25 <= i < 30:
        career.append(7)
    else:
        career.append(8)

# 근무경력수치 feature
X['근무경력수치'] = career

# 근무경력수치 * 세부직종_order
X['근무경력수치*세부직종order'] = X['근무경력수치'] * X['세부직종_order']

- 대학별 근무경력구분

In [22]:
univ_lst = X['출신대학'].unique().tolist()
univ_people = []
univ_career = []


for univ in univ_lst:
    univ_people_cnt = len(X['근무경력_y'][X['출신대학'] == univ])
    mean_career = X['근무경력_y'][X['출신대학'] == univ].mean()
    
    
    univ_people.append(univ_people_cnt)
    univ_career.append(mean_career)
    
univ_mean_salary = pd.DataFrame({'출신대학' : univ_lst,
                                 '출신인원' : univ_people,
                                 '평균경력' : univ_career})

In [23]:
high = []
middle = [] 
low = []

for val in univ_mean_salary['평균경력']:
    if val >= 3:
        high.append(univ_mean_salary['출신대학'][univ_mean_salary['평균경력'] == val])
    elif 2.5 <= val < 3:
        middle.append(univ_mean_salary['출신대학'][univ_mean_salary['평균경력'] == val])
    else:
        low.append(univ_mean_salary['출신대학'][univ_mean_salary['평균경력'] == val])
    
high_lst = []
middle_lst = []
low_lst = []

for cnt in range(len(high)):
    high_lst.append(np.array(high)[cnt][0])

for cnt in range(len(middle)):
    middle_lst.append(np.array(middle)[cnt][0])

for cnt in range(len(low)):
    low_lst.append(np.array(low)[cnt][0])   
    
X['대학별경력구분'] = X['출신대학'].copy()

def univ_career(x):
    if x in high_lst:
        return 'high'
    elif x in middle_lst:
        return 'middle'
    elif x in low_lst:
        return 'low'

X['대학별경력구분'] = X['대학별경력구분'].map(lambda x: univ_career(x))

In [24]:
X['대학별경력구분'].head(5)

0      high
1       low
2    middle
3    middle
4       low
Name: 대학별경력구분, dtype: object

- 근무지역_분류

In [25]:
X_ = X['근무지역'].copy()

lst = []
for val in X_.map(lambda x: x.split(',')):
    for k in val:
        if k not in lst:
            lst.append(k)
            
수도권 = ['서울', '경기', '인천']
지방 = ['강원', '경남', '경북', '충남', '충북', '전남', '전북', '제주', '세종']
광역시 = ['대구', '부산', '광주', '울산', '대전',]
해외 = ['일본', '인도', '미국', '해외', '캐나다', '말레이시아', '홍콩', '중국', '인도네시아', '대만', '싱가포르', '방글라데시', '프랑스', '필리핀', '러시아']
기타 = ['기타', '전국']

X_ = X_.map(lambda x: x.split(','))

for row in range(len(X_)):
    for idx in range(len(val)):
        if  X_[row][idx] in 수도권:
            X_[row][idx] = '수도권'
        elif X_[row][idx] in 지방:
            X_[row][idx] = '지방'
        elif X_[row][idx] in 광역시:
            X_[row][idx] = '광역시'
        elif X_[row][idx] in 해외:
            X_[row][idx] = '해외'
        else:
            X_[row][idx] = '기타'
            
for row in range(len(X_)):
    X_[row].sort()
    
X_ = X_.map(lambda x: ','.join(x))
X_ = pd.DataFrame(X_)
X['근무지역_분류'] = X_['근무지역']

- 근무지역_분류를 기반으로 근무지역_count

In [26]:
# 근무지역_count
region_dict = {'해외':5, '수도권':4, '광역시':3, '지방':2, '기타':1}
region_df = X[['근무지역_분류']].copy()
region_df['근무지역_분류'] = region_df['근무지역_분류'].map(lambda x: x.split(',')) 

def region_transformer(x):
    for idx in range(len(x)):
        x[idx] = region_dict[x[idx]]
        
    return sum(x)

region_df['근무지역_count'] = region_df['근무지역_분류'].map(lambda x: region_transformer(x))

X['근무지역_count'] = region_df['근무지역_count']

### 파생 features 
- v20.4, v21.4와의 차이점
1. '근무경력_y' -> '근무지역_d'로 변경
2. '숙련도_세직_order' & '숙련도_해외근무경력' feature 추가

In [27]:
X['근무경력_d*세부order'] = X['근무경력_d'] * X['세부직종_order']
X['해외근무경력_세직_order'] = X['해외근무경력'] + '_' + X['세부직종_order'].astype(str)
X['대학+전공'] = X['출신대학'] + X['대학전공']
X['직종+세부직종']= X['직종'] + X['세부직종']
X['근무형태+직종'] = X['근무형태']+X['직종']
X['출신대학수치*근무경력_d'] = X['출신대학수치'] * X['근무경력_d']
X['출신대학수치*세부직종_order'] = X['출신대학수치'] * X['세부직종_order']
X['세부직종+근무지역'] = X['세부직종'] + X['근무지역']
X['근무형태+세부직종'] = X['근무형태'] + X['세부직종']
X['근무형태+출신대학'] = X['근무형태'] + X['출신대학']
X['근무형태+근무지역'] = X['근무형태'] + X['근무지역']
X['근무형태+대학전공'] = X['근무형태'] + X['대학전공']
X['근무형태+직무태그_pr'] = X['근무형태'] + X['직무태그_pr']
X['근무형태_숙련도'] = X['근무형태'] + '_' + X['숙련도']
X['세직_태그_pr'] = X['세부직종'] + '_' + X['직무태그_pr']
X['숙련도_세직_order'] = X['숙련도'] + '-' + X['세부직종_order'].astype(str)
X['숙련도_해근경'] = X['숙련도'] + '-' + X['해외근무경력']
X['지역_cnt*세직_order'] = X['근무지역_count'] * X['세부직종_order']
X['지역_cnt*근무경력_d'] = X['근무지역_count'] * X['근무경력_d']

In [28]:
X.drop('근무지역_분류',axis=1,inplace=True)

# CATBOOST MODEL

In [29]:
X_train = X.iloc[:X_train.shape[0],:].reset_index(drop=True)
X_test = X.iloc[X_train.shape[0]:,:].reset_index(drop=True)

In [30]:
numeric_features = X_train.dtypes[X_train.dtypes != "object"].index.tolist()
print("Number of Numerical features: ", len(numeric_features))

categorical_features = X_train.dtypes[X_train.dtypes == "object"].index.tolist()
print("Number of Categorical features: ", len(categorical_features))

Number of Numerical features:  33
Number of Categorical features:  28


In [31]:
numeric_features

['대학성적',
 '근무경력_y',
 '근무경력_m',
 '근무경력_d',
 '대학_평균근무경력',
 '대학_근무경력std',
 '대학_근무경력var',
 '세부직종_평균근무경력',
 '세부직종_근무경력std',
 '세부직종_근무경력var',
 '직종_평균근무경력',
 '직종_근무경력std',
 '직종_근무경력var',
 '근무형태_평균근무경력',
 '근무형태_근무경력std',
 '근무형태_근무경력var',
 '대학전공_평균근무경력',
 '대학전공_근무경력std',
 '대학전공_근무경력var',
 '직무태그_pr_평균근무경력',
 '직무태그_pr_근무경력std',
 '직무태그_pr_근무경력var',
 '세부직종_order',
 '출신대학수치',
 '출신대학수치*대학성적',
 '근무경력수치',
 '근무경력수치*세부직종order',
 '근무지역_count',
 '근무경력_d*세부order',
 '출신대학수치*근무경력_d',
 '출신대학수치*세부직종_order',
 '지역_cnt*세직_order',
 '지역_cnt*근무경력_d']

In [32]:
# binary features와 중복되는 feature 제외
categorical_features.remove('직무태그')
categorical_features.remove('근무지역')
categorical_features.remove('근무형태')
categorical_features

['직종',
 '세부직종',
 '출신대학',
 '대학전공',
 '어학시험',
 '자격증',
 '숙련도',
 '해외근무경력',
 '직무태그_pr',
 '대학별그룹',
 '대학별경력구분',
 '해외근무경력_세직_order',
 '대학+전공',
 '직종+세부직종',
 '근무형태+직종',
 '세부직종+근무지역',
 '근무형태+세부직종',
 '근무형태+출신대학',
 '근무형태+근무지역',
 '근무형태+대학전공',
 '근무형태+직무태그_pr',
 '근무형태_숙련도',
 '세직_태그_pr',
 '숙련도_세직_order',
 '숙련도_해근경']

In [33]:
categorical_features = categorical_features
binary_features = ['직무태그','근무지역','근무형태']

X_train = X_train[numeric_features+categorical_features+binary_features]  # 순서 주의!!!
X_test = X_test[numeric_features+categorical_features+binary_features]

# CatBoost의 cat_features 파라미터에 지정할 범주형 피처 위치
cat_index = [list(X_train.columns).index(c) for c in categorical_features]

In [34]:
NCOMP = 50
P = 0.05

In [35]:
# 상하한값 제한을 통한 결측값 처리 함수: FunctionTransformer를 통해 호출
def remove_outlier(X, q=0.01):  
    df = pd.DataFrame(X)
    return df.apply(lambda x: x.clip(x.quantile(q), x.quantile(1-q)), axis=0).values

# 회귀분석의 계수검정을 이용한 피처선택 전처리기 클래스
class MyFeatureSelector(TransformerMixin, BaseEstimator):
    # 전처리기 생성 즉, MyFeatureSelector() 호출시 실행
    def __init__(self, p=0.01):
        self.p = p

    # 전처리기의 fit() 호출시 실행
    def fit(self, X, y=None):
        X = sm.add_constant(X)
        results = sm.OLS(y, X).fit()
        self.cols = list(results.pvalues[1:] <= self.p)
        return self
    
    # 전처리기의 transform() 호출시 실행
    def transform(self, X):
        return X[:,self.cols].astype(np.int64)        
    
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("outlier", FunctionTransformer(remove_outlier, kw_args={'q':0.01})),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")), 
        ("encoder", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=99999, dtype=np.object)),
    ]
)

binary_transformer = Pipeline(
    steps=[
        ("imputer", FunctionTransformer(lambda x: x.fillna('없음'))),      
        ("corpus", FunctionTransformer(lambda x: x.str.replace('·',',').str.split(',').str.join(" "))),
        ("BoW", CountVectorizer()),
        ("dense", FunctionTransformer(lambda x: x.toarray().astype(int), accept_sparse=True)),
    ]
)

column_transformer = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
        ("bin1", make_pipeline(binary_transformer, TruncatedSVD(n_components=NCOMP,random_state=0)), binary_features[0]),
        ("bin2", make_pipeline(binary_transformer, MyFeatureSelector(p=P)), binary_features[1]),
        ("bin3", make_pipeline(binary_transformer, MyFeatureSelector(p=P)), binary_features[2])
    ]
)

preprocessor = Pipeline(
    steps=[
        ("column", column_transformer), 
    ]
)

In [36]:
X_train = preprocessor.fit_transform(X_train, y_train)
X_test = preprocessor.transform(X_test)

In [37]:
%%time

# 최적화된 하이퍼파라미터로 OOF를 수행하여 최종 CatBoost 모형 생성:
# No tuning => tuning한 모델에 비해 성능이 떨어지지 않음

# sscv = ShuffleSplit(test_size=.3334, n_splits=15, random_state=0)
models = cross_validate(CatBoostRegressor(cat_features=cat_index, verbose=False, random_state=0),
                        X_train, y_train, 
                        cv=15, 
                        scoring='neg_mean_squared_error', 
                        return_estimator=True, n_jobs=-1)
oof_pred = np.array([m.predict(X_test) for m in models['estimator']]).mean(axis=0)

CPU times: total: 5.94 s
Wall time: 5min 30s


In [38]:
scores = models['test_score']
print("\nCatBoost CV scores: ", np.sqrt(-1*scores))
print("CatBoost CV mean = %.2f" % np.sqrt(-1*scores.mean()), "with std = %.2f" % np.sqrt(scores.std()))

# # submission 화일 생성
# CATBOOST_VERSION = 22.0
# filename = f'catboost_{CATBOOST_VERSION}_{np.sqrt(-1*scores.mean()):.2f}.csv'
# path = './submission\\' 
# pd.DataFrame({'ID':test_id, 'Salary':oof_pred}).to_csv(path + filename, index=False)


CatBoost CV scores:  [716.79979142 769.51756119 736.47126064 727.19805649 774.16174846
 706.21626065 805.13894187 827.86589557 828.36026989 783.2015978
 875.53958915 933.85464845 881.04484577 931.09321734 981.12118828]
CatBoost CV mean = 822.71 with std = 372.99
